In [ ]:
import requests
import pandas as pd
import time
import os
from concurrent.futures import ThreadPoolExecutor

symbols = [
    "DIS",
    "BAC",
    "WMT",
    "ADBE",
    "PFE",
    "KO",
    "PEP",
    "INTC",
    "CSCO",
    "XOM",
    "CVX",
    "ORCL",
    "T",
    "MRK",
    "UNH",
    "TGT",
    "QCOM",
    "COST",
    "UPS",
    "PYPL",
    "MMM",
    "BA",
    "IBM",
    "AMD",
    "GE",
    "SBUX",
    "MDT",
    "NKE",
    "HON",
    "RTX",
]

start_date = "2022-01-01"
end_date = "2024-12-31"

output_file = "combined_stock_data.csv"


def fetch_technical_indicator(symbol, indicator, interval="daily", time_period=20):
    """
    Gets a specific technical indicator (like SMA or RSI) for a stock.

    Args:
        symbol: The stock ticker, like 'AAPL'.
        indicator: The name of the technical indicator (e.g., 'SMA', 'RSI').
        interval: How often the data is reported, usually 'daily'. Defaults to 'daily'.
        time_period: The number of data points to calculate the indicator. Defaults to 20.

    Returns:
        dict: A dictionary with the indicator data, or an empty dictionary if something went wrong.
    """
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": indicator,
        "symbol": symbol,
        "interval": interval,
        "time_period": time_period,
        "series_type": "close",
        "apikey": API_KEY,
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching {indicator} for {symbol}: HTTP {response.status_code}")
        return {}


def fetch_daily_data(symbol):
    """
    Pulls daily stock prices for a specific stock.

    Args:
        symbol: The stock ticker, like 'AAPL'.

    Returns:
        pd.DataFrame: A dataFrame with the daily price, volume data etc.
                      If there's an issue, you'll get an empty table.
    """
    url = f"https://www.alphavantage.co/query"
    params = {"function": "TIME_SERIES_DAILY", "symbol": symbol, "outputsize": "full"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if "Time Series (Daily)" in data:
            time_series = data["Time Series (Daily)"]
            df = pd.DataFrame.from_dict(time_series, orient="index")
            df.reset_index(inplace=True)
            df.columns = ["date", "open", "high", "low", "close", "volume"]
            df["date"] = pd.to_datetime(df["date"])
            df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
            df[["open", "high", "low", "close", "volume"]] = df[
                ["open", "high", "low", "close", "volume"]
            ].astype(float)
            df["symbol"] = symbol  # Add the symbol column
            return df
        else:
            print(
                f"Error fetching data for {symbol}: {data.get('Note', 'Unknown error')}"
            )
    else:
        print(f"HTTP Error {response.status_code} for {symbol}")
    return pd.DataFrame()


def fetch_stock_data(symbol):
    """
    Combines daily stock prices and technical indicators for a stock.

    Args:
        symbol: The stock ticker, like 'AAPL'.

    Returns:
        pd.DataFrame: A dataframe with daily stock prices and indicators combined.
                      If no data is available, you'll get an empty table.
    """
    print(f"Fetching data for {symbol}...")
    stock_data = fetch_daily_data(symbol)
    if stock_data.empty:
        print(f"No stock data available for {symbol}")
        return pd.DataFrame()

    indicators = ["SMA", "EMA", "MACD", "RSI", "BBANDS", "ATR"]
    all_indicators = {}
    for indicator in indicators:
        data = fetch_technical_indicator(symbol, indicator)
        if f"Technical Analysis: {indicator}" in data:
            ind_df = pd.DataFrame.from_dict(
                data[f"Technical Analysis: {indicator}"], orient="index"
            )
            ind_df.reset_index(inplace=True)
            ind_df.columns = ["date"] + list(ind_df.columns[1:])
            ind_df["date"] = pd.to_datetime(ind_df["date"])
            all_indicators[indicator] = ind_df

    for indicator, ind_df in all_indicators.items():
        stock_data = stock_data.merge(ind_df, on="date", how="left")

    return stock_data


def main():
    """
    Fetches stock data for all the symbols in the list and saves it to a CSV file.

    This function runs everything concurrently, so it processes multiple stocks at once to save time.
    If successful, the data is saved to 'combined_stock_data.csv'.
    """
    combined_data = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = executor.map(fetch_stock_data, symbols)
        for result in results:
            if not result.empty:
                combined_data.append(result)

    if combined_data:
        final_df = pd.concat(combined_data, ignore_index=True)
        final_df.to_csv(output_file, index=False)
        print(f"Combined data saved to {output_file}")
    else:
        print("No data fetched for any stock.")


if __name__ == "__main__":
    main()